In [12]:
import requests
import os
import re
import pandas as pd
import sklearn
import nltk
import string
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [13]:
client_id = "yxeLnXDdwoDJunEPDRUgYVmH-I703sGZKWGoCdKbFNupYKHugalkOVqtzh3aSe82"
client_secret = "zd3utRAeYAprwImrgEPERa6BPjNrJMOcg4ogsbTAR-uCoIoh9zu28ufhMVRYlBTpb8kSHkWK75LDrHSerdEOag"
client_access_token = "wVsdPF-_vbLLypQZ3qaR0EwytwJW2jKBsK-PoZwv6BDKQLDK0gkUwstOYuH-EDD2"

In [81]:
def request_song_info(artist_name, song_title):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': f'Bearer {client_access_token}'}
    search_url = base_url + '/search'
    params = {'q': f'{artist_name} {song_title}'}

    response = requests.get(search_url, params = params, headers = headers)
    return response

def request_song_url(artist_name, song_titles):
    # Input is song titles, want to find associated urls with each
    urls = []
    track_names = []

    for song in song_titles:
        #print(song)
        song = re.sub("´", "'", song)
        song = re.sub("  ", " ", song)
        response = request_song_info(artist_name, song)
        json = response.json()
    
        # Extract the first matching song URL
        hit = json['response']['hits'][0]['result']
        #print(hit)
        #print(hit['title'].lower())
        if (artist_name.lower() in hit['primary_artist']['name'].lower()):
            song_url = hit['url']
            urls.append(song_url)
            
            match = re.search(r'beatles-(.*?)-lyrics', song_url)
    
            if match:
                result = match.group(1) 
                track_names.append(result)
        
    # Return both the URL and the song title
    return [urls, track_names] 

In [80]:
# First want to only have songs that are composed by lennon or mccartney
os.chdir("C:/Users/maede/Downloads/Python/INFO5653/Project/Project_Part1/INFO5653_Project")
df = pd.read_csv("song_composer_singer.csv")
df.columns = ["Song", "Composer", "Singer"]
lennon_mccartney = df[df['Composer'].isin(["Lennon", "McCartney"])]

# Now want to match songs from this set to the lyrics pulled from genuis api
l_m_songs = list(lennon_mccartney['Song'])

# Composer list
l_m_composer = list(lennon_mccartney['Composer'])

# Find urls!
artist_name = "The Beatles"
[urls, song_titles] = request_song_url(artist_name, l_m_songs)

{'meta': {'status': 200}, 'response': {'hits': [{'highlights': [], 'index': 'song', 'type': 'song', 'result': {'annotation_count': 3, 'api_path': '/songs/117716', 'artist_names': 'The Beatles', 'full_title': 'Love Me Do by\xa0The\xa0Beatles', 'header_image_thumbnail_url': 'https://images.rapgenius.com/956cd17ed482217b2148380468e1ff30.300x300x1.jpg', 'header_image_url': 'https://images.rapgenius.com/956cd17ed482217b2148380468e1ff30.360x360x1.jpg', 'id': 117716, 'lyrics_owner_id': 70799, 'lyrics_state': 'complete', 'path': '/The-beatles-love-me-do-lyrics', 'primary_artist_names': 'The Beatles', 'pyongs_count': 13, 'relationships_index_url': 'https://genius.com/The-beatles-love-me-do-sample', 'release_date_components': {'year': 1964, 'month': 3, 'day': 22}, 'release_date_for_display': 'March 22, 1964', 'release_date_with_abbreviated_month_for_display': 'Mar. 22, 1964', 'song_art_image_thumbnail_url': 'https://images.rapgenius.com/956cd17ed482217b2148380468e1ff30.300x300x1.jpg', 'song_art_

KeyboardInterrupt: 

In [65]:
len(song_titles)

143

In [61]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    song = requests.get(url)
    html = BeautifulSoup(song.content, 'html.parser')
    lyrics_divs = html.find_all("div", {"data-lyrics-container": "true"})
    
    if lyrics_divs:
        lyrics = "\n".join(div.get_text(separator="\n") for div in lyrics_divs)
        # Remove identifiers like chorus, verse, etc
        lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics, flags=re.DOTALL).strip()
        
        # Remove empty lines
        lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])  
    else:
        print("Lyrics not found!")
        lyrics = ""
    
    return lyrics

In [69]:
def write_lyrics_to_corpus(urls, track_names, composer, path):
    for i in range(len(track_names)):
        # Get url and title
        url = urls[i]
        title = track_names[i]
        label = composer[i]
        title = re.sub(" ", "-", title)
        title = re.sub(r"\?", "", title)
        title = title.lower()
    
        # Create file, mypath, and filename
        file = str(label) + '-' + str(title) + ".txt"
        filename = mypath + "/" + file
    
        # Open file to write to
        my_file = open(filename, "w", encoding = "utf-8")
    
        # Find lyrics
        lyrics = scrape_song_lyrics(url)
        my_file.write(lyrics)
        my_file.close()

mypath = "C:/Users/maede/Downloads/Python/INFO5653/Project/Project_Part1/INFO5653_Project/Composer_Lyrics_Corpus"
write_lyrics_to_corpus(urls, song_titles, l_m_composer, mypath)

In [70]:
########## PART 3: USE CORPUS TO BUILD DATAFRAME ##########

# Path to corpus
path = "C:/Users/maede/Downloads/Python/INFO5653/Project/Project_Part1/INFO5653_Project/Composer_Lyrics_Corpus"

# Create file name
file_name_list = os.listdir(path)[1:]

# Initliaze empty lists that will be filled later with all file paths in corpus
complete_file_paths = [] 
list_file_names = []
    
for name in file_name_list:
    nextfile = path + "/" + name 
    complete_file_paths.append(nextfile) 
    nextnameL = name.split(".")
    list_file_names.append(nextnameL[0])

In [71]:
# USE COUNTVECTORIZOR AND LET MAX_FEATURES = 100

# Now we can use CountVectorizer 
my_vect = CountVectorizer(input = 'filename', stop_words = 'english', token_pattern=r'\b[a-zA-Z]{4,}\b', max_features = 100)
fit = my_vect.fit_transform(complete_file_paths)
column_names = my_vect.get_feature_names_out()

# Convert to dataframe
lyrics_df = pd.DataFrame(fit.toarray(),columns = column_names)

# Add labels using a dictionary
dict_labels = {}

for i in range(0, len(complete_file_paths)):
    dict_labels[i] = list_file_names[i].split("-")[0]

lyrics_df = lyrics_df.rename(dict_labels, axis = "index")
lyrics_df.index.name = "LABEL"

# Write dataframe to csv
lyrics_df.to_csv("paul_john_lyrics_corpus_count100.csv", index = True)

In [72]:
# USE TfidfVectorizer AND LET MAX_FEATURES = 100
# 100
my_vect = TfidfVectorizer(input = 'filename', stop_words = 'english', token_pattern=r'\b[a-zA-Z]{4,}\b', max_features = 100)
fit = my_vect.fit_transform(complete_file_paths)
column_names = my_vect.get_feature_names_out()

# Convert to dataframe
lyrics_df = pd.DataFrame(fit.toarray(),columns = column_names)

# Add labels using a dictionary
dict_labels = {}

for i in range(0, len(complete_file_paths)):
    dict_labels[i] = list_file_names[i].split("-")[0]

lyrics_df = lyrics_df.rename(dict_labels, axis = "index")
lyrics_df.index.name = "LABEL"

# Write dataframe to csv
lyrics_df.to_csv("paul_john_lyrics_corpus_tfid100.csv", index = True)

In [73]:
# LEMMING, CountVectorizer, max_features = 100
LEMMER = WordNetLemmatizer() 

def MY_LEMMER(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split()
    words = [LEMMER.lemmatize(word) for word in words]
    return words
    
my_vect = CountVectorizer(input = 'filename', tokenizer = MY_LEMMER, max_features = 100)
fit = my_vect.fit_transform(complete_file_paths)
column_names = my_vect.get_feature_names_out()

# Convert to dataframe
lyrics_df = pd.DataFrame(fit.toarray(),columns = column_names)

# Add labels using a dictionary
dict_labels = {}

for i in range(0, len(complete_file_paths)):
    dict_labels[i] = list_file_names[i].split("-")[0]

lyrics_df = lyrics_df.rename(dict_labels, axis = "index")
lyrics_df.index.name = "Label"

# Write dataframe to csv
lyrics_df.to_csv("paul_john_lyrics_corpus_lem_count100.csv", index = True)

C:\Users\maede\New folder\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [74]:
# LEMMING, TfidfVectorizer, max_features = 100
LEMMER = WordNetLemmatizer() 

def MY_LEMMER(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split()
    words = [LEMMER.lemmatize(word) for word in words]
    return words
    
my_vect = TfidfVectorizer(input = 'filename', tokenizer = MY_LEMMER, max_features = 100)
fit = my_vect.fit_transform(complete_file_paths)
column_names = my_vect.get_feature_names_out()

# Convert to dataframe
lyrics_df = pd.DataFrame(fit.toarray(),columns = column_names)

# Add labels using a dictionary
dict_labels = {}

for i in range(0, len(complete_file_paths)):
    dict_labels[i] = list_file_names[i].split("-")[0]

lyrics_df = lyrics_df.rename(dict_labels, axis = "index")
lyrics_df.index.name = "Label"

# Write dataframe to csv
lyrics_df.to_csv("paul_john_lyrics_corpus_lem_tfid100.csv", index = True)

C:\Users\maede\New folder\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [75]:
# STEMMING, CountVectorizer, max_features = 100
STEMMER = PorterStemmer()
def MY_STEMMER(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split()
    words = [STEMMER.stem(word) for word in words]
    return words
    
my_vect = CountVectorizer(input = 'filename', tokenizer = MY_STEMMER, max_features = 100)
fit = my_vect.fit_transform(complete_file_paths)
column_names = my_vect.get_feature_names_out()

# Convert to dataframe
lyrics_df = pd.DataFrame(fit.toarray(),columns = column_names)

# Add labels using a dictionary
dict_labels = {}

for i in range(0, len(complete_file_paths)):
    dict_labels[i] = list_file_names[i].split("-")[0]

lyrics_df = lyrics_df.rename(dict_labels, axis = "index")
lyrics_df.index.name = "Label"

# Write dataframe to csv
lyrics_df.to_csv("paul_john_lyrics_corpus_stem_count100.csv", index = True)

C:\Users\maede\New folder\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [76]:
# STEMMING, TfidfVectorizer, max_features = 100
STEMMER = PorterStemmer()
def MY_STEMMER(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split()
    words = [STEMMER.stem(word) for word in words]
    return words
    
my_vect = TfidfVectorizer(input = 'filename', tokenizer = MY_STEMMER, max_features = 100)
fit = my_vect.fit_transform(complete_file_paths)
column_names = my_vect.get_feature_names_out()

# Convert to dataframe
lyrics_df = pd.DataFrame(fit.toarray(),columns = column_names)

# Add labels using a dictionary
dict_labels = {}

for i in range(0, len(complete_file_paths)):
    dict_labels[i] = list_file_names[i].split("-")[0]

lyrics_df = lyrics_df.rename(dict_labels, axis = "index")
lyrics_df.index.name = "Label"

# Write dataframe to csv
lyrics_df.to_csv("paul_john_lyrics_corpus_stem_tfid100.csv", index = True)

C:\Users\maede\New folder\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
